In [ ]:
import sys, os
sys.path.append(os.path.abspath(".."))

from pycfx.datasets import *
from pycfx.models import *
from pycfx.benchmarker import *
from pycfx.helpers.visualisation import *
from pycfx.conformal import *
from pycfx.counterfactual_explanations import *

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
Dataset = SyntheticBimodal
factual = np.array([1, 2.5])
y_target = 1 

In [ ]:
dataset = Dataset(0.6, 0.2, 0.2)
X_train, y_train, X_calib, y_calib, X_test, y_test = dataset.get_X_y_split()
plot_split_dataset(X_train, X_test, y_train, y_test)

In [ ]:
model = PyTorchMLP(config={}, input_properties=dataset.input_properties)
model.load_or_train(Path("facedata2"), X_train, y_train, True)

In [ ]:
plot_decision_boundary(model, X_calib, X_calib, y_calib, y_calib)

In [ ]:
#Min distance CF
cf_gen = MinDistanceCF(model, dataset.input_properties, config={"db_distance":0.05})
counterfactual = cf_gen.generate_counterfactual(factual, y_target)
plot_counterfactual(factual, counterfactual, model, X_calib, X_calib, y_calib, y_calib, faded_background=True)

In [ ]:
conformalCF = ConformalCF(model, dataset.input_properties, config={"conformal_class": SplitConformalPrediction, "conformal_config": {"alpha": 0.03}})
conformalCF.setup(X_train, y_train, X_calib, y_calib)
counterfactual_conformal = conformalCF.generate_counterfactual(factual, y_target)
plot_counterfactual(factual, counterfactual_conformal, model, X_calib, X_calib, y_calib, y_calib, conformal=conformalCF.conformal)

In [ ]:
ccf = {"alpha": 0.03,
        "scorefn_name":'linear',
        "kernel_name":'box_l1', 
        "kernel_bandwidth": 0.35}

In [ ]:
conformal_cfxt = BaseLCP(model, dataset.input_properties, ccf)
conformal_cfxt.calibrate(X_calib, y_calib, None)
plot_conformal_prediction(model, conformal_cfxt, X_calib, X_calib, y_calib, y_calib)

In [ ]:
conformalCF_b = ConformalCF(model, dataset.input_properties, config={"conformal_class": BaseLCP, "conformal_config": ccf})
conformalCF_b.setup(X_train, y_train, X_calib, y_calib)
counterfactual_conformal_b = conformalCF_b.generate_counterfactual(factual, y_target)
plot_counterfactual(factual, counterfactual_conformal_b, model, X_calib, X_calib, y_calib, y_calib, conformal=conformalCF_b.conformal)

In [ ]:
ccf2 = {"alpha": 0.05,
        "scorefn_name":'linear2',
        "kernel_name":'box_l1', 
        "kernel_bandwidth": 0.3,
        "inf_quantile": False,
        "global_quantile": False,
        "split_midpoint": True,
        }

conformal_cfxt = ConformalCONFEXTree(model, dataset.input_properties, ccf2)
conformal_cfxt.calibrate(X_calib, y_calib, None)
plot_conformal_prediction(model, conformal_cfxt, X_calib, X_calib, y_calib, y_calib)

In [ ]:
wachter = WachterGenerator(model, dataset.input_properties, config={"mad": True})
wachter.setup(X_train, y_train, X_calib, y_calib)
cfwachter = wachter.generate_counterfactual(factual, y_target)
plot_counterfactual(factual, cfwachter, model, X_calib, X_calib, y_calib, y_calib)

In [ ]:
conformalCF_c = ConformalCF(model, dataset.input_properties, config={"conformal_class": ConformalCONFEXTree, "conformal_config": ccf2})
conformalCF_c.setup(X_train, y_train, X_calib, y_calib)
counterfactual_conformal_c = conformalCF_c.generate_counterfactual(factual, y_target)
plot_counterfactual(factual, counterfactual_conformal_c, model, X_calib, X_calib, y_calib, y_calib, conformal=conformalCF_c.conformal)